In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
past_teams = pd.read_csv('../../data/indices/web_scraped_past.csv').columns.to_list()

ids = pd.read_csv('../../data/indices/web_scraped_shortened.csv').values.tolist()
ids = result_dict = {name: index for index, name in ids}

years = ["2010", "2011", "2012", "2014", "2015", "2016", "2017", "2018", "2020", "2021", "2022", "2023"]

In [3]:
def swap_ids_and_count_games(year):
    games = pd.read_csv(f'../../data/dataframes/game/games_{year}.csv')
    games = games.sort_values(by=['TEAM', 'DATE']).reset_index(drop=True)
    games.insert(1, "GAME", 0)
    games.insert(3, "TEAM_ID", 0)
    games.insert(5, "OPP_ID", 0)

    ctr = 0
    curr = games.loc[0, "TEAM"]

    for team in past_teams:
        mask = (games["TEAM"] == team)
        games = games[~mask]
        mask = (games["OPP."] == team)
        games = games[~mask]

    games = games.reset_index(drop=True)

    for i in range(len(games)):
        if curr == games.loc[i, "TEAM"]:
            games.loc[i, "GAME"] = ctr
            games.loc[i, "TEAM_ID"] = ids[curr]
            games.loc[i, "OPP_ID"] = ids[games.loc[i, "OPP."]]
            games.loc[i, "VENUE"] = ord(games.loc[i, "VENUE"])
            ctr += 1
        else:
            ctr = 1
            curr = games.loc[i, "TEAM"]
            games.loc[i, "TEAM_ID"] = ids[curr]
            games.loc[i, "OPP_ID"] = ids[games.loc[i, "OPP."]]
            games.loc[i, "VENUE"] = ord(games.loc[i, "VENUE"])

    games = games.drop(columns=['TEAM', 'OPP.'])
    games.to_csv(f"../../data/finals/game_stats/final_games_{year}_wdates.csv", index=False)

In [4]:
for year in years:
    swap_ids_and_count_games(year)